In [167]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [1]:
from tqdm.notebook import tqdm as tqdm
import requests
from bs4 import BeautifulSoup as bs4
import openpyxl as opx
from lxml import html
from time import sleep
import re

Gammas = []
Xrays = []
class ToRI:
    def __init__(self):
        self.Gammas_table = []
        self.Xrays_table = []
        self.X_table = []
        self.Gammas = []
        self.stable = []
        self.Read_Gammas()
#         self.Mold()
#         self.ToRI_scraping()
#         self.write_Gammas('finish')
#         self.write_Xrays('finish')
        

    def ToRI_scraping(self):
        for z in tqdm(range(79,110)):
#             z = 13
            sleep(5)
            url = 'http://nucleardata.nuclear.lu.se/toi/listnuc.asp?Z='+str(z)
            page_res = requests.get(url).text
            page_soup = bs4(page_res, 'lxml')
            self.a_url = page_soup.find_all('a')[:-3]
            for i in tqdm(self.a_url):
                sleep(5)
                check_url = i.attrs['href']
                check_url = 'http://nucleardata.nuclear.lu.se/toi/' + check_url
                page2_res = requests.get(check_url).text
                page2_soup = bs4(page2_res, 'lxml')
                per_table = page2_soup.find_all('table')
                per_table_level = 0
                if 'level' in per_table[0].text.replace(u'\xa0', ' '):
                    per_table_level = per_table[0].text.replace(u'\xa0', ' ').split('\n')[19].split(':')[1].split()[0]
                else:
                    per_table_level = 0
                for ii in per_table:
                    per_table = ii.text.replace(u'\xa0', ' ')
                    if per_table.startswith('\nGammas from'):
                        per_table_Gammas = per_table.split('\n')
                        num = re.sub('\\D','',per_table_Gammas[1].split()[2])
                        for ii in range((len(per_table_Gammas)-5)//5):
                            if per_table_Gammas[7+5*ii] == ' ':
                                self.Gammas_table.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+'no data'+','\
                                                         +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                                Gammas.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+'no data'+','\
                                                         +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                            else:
                                if len(per_table_Gammas[7+5*ii].split()) == 2 and '*' in per_table_Gammas[7+5*ii].split()[1]:
                                    self.Gammas_table.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+per_table_Gammas[7+5*ii].split()[0]+'*'+','\
                                                         +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                                    Gammas.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+per_table_Gammas[7+5*ii].split()[0]+'*'+','\
                                                         +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                                else:
                                    self.Gammas_table.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+per_table_Gammas[7+5*ii].split()[0]+','\
                                                             +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                                    Gammas.append(str(z)+','+num+','+str(per_table_level)+','+per_table_Gammas[6+5*ii].split()[0]+','+per_table_Gammas[7+5*ii].split()[0]+','\
                                                     +(per_table_Gammas[1].split()[3]+per_table_Gammas[1].split()[4]).lstrip('('))
                    if per_table.startswith('\nX-rays from'):
                        per_table_Xrays = per_table.split('\n')
                        num = re.sub('\\D','',per_table_Xrays[1].split()[2])
                        for ii in range((len(per_table_Xrays)-5)//5):
                            energy = per_table_Xrays[6+5*ii].split()[0].replace(',','.')
                            if per_table_Xrays[7+5*ii] == ' ':
                                self.Xrays_table.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+'no data'+','\
                                                         +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                                Xrays.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+'no data'+','\
                                                         +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                            else:
                                if len(per_table_Xrays[7+5*ii].split()) == 2 and '*' in per_table_Xrays[7+5*ii].split()[1]:
                                    self.Xrays_table.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+per_table_Xrays[7+5*ii].split()[0]+'*'+','\
                                                         +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                                    Xrays.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+per_table_Xrays[7+5*ii].split()[0]+'*'+','\
                                                         +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                                else:
                                    self.Xrays_table.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+per_table_Xrays[7+5*ii].split()[0]+','\
                                                             +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                                    Xrays.append(str(z)+','+num+','+str(per_table_level)+','+energy+','+per_table_Xrays[7+5*ii].split()[0]+','\
                                                     +(per_table_Xrays[1].split()[3]+per_table_Xrays[1].split()[4]).lstrip('('))
                                    
            self.write_Gammas(z)
            self.write_Xrays(z)

    
    def write_Gammas(self, x):
        with open("Gammas"+str(x)+".csv", mode = "w") as f:
            for i in self.Gammas_table:
                f.write(i)
                f.write("\n")
        f.close
        
    def write_Xrays(self, x):
        with open("Xrays"+str(x)+".csv", mode = "w") as f:
            for i in self.Xrays_table:
                f.write(i)
                f.write("\n")
        f.close
        
    def Read_Gammas(self):
        f = open("Xrays_molded.csv")
        line = f.readline()
        while line:
            line = line.rstrip()
            self.Gammas.append(line.rstrip().split(','))
            if float(line.rstrip().split(',')[1]) > 500:
                print(line)
            line = f.readline()
        f.close
    
    def Mold(self):
        with open("Xrays_molded.csv", mode = "w") as f:
            for i in self.Gammas:
                if i[4] != "no data":
                    if i[5][0] == "~":
                        i[5] = i[5][1:]
                        if str(i[5][len(i[5])-2]) + str(i[5][len(i[5])-1]) == "ms":
                            d = ""
                            for ii in range(len(i[5])-2):
                                d = d + i[5][ii]
                            d = float(d)
                            f.write(i[0]+","+i[1]+","+i[2]+","+i[3]+","+i[4]+","+"~"+str(d/1000)+"s"+"\n")
                        else:
                            f.write(i[0]+","+i[1]+","+i[2]+","+i[3]+","+i[4]+","+i[5]+"\n")
                    else:
                        if str(i[5][len(i[5])-2]) + str(i[5][len(i[5])-1]) == "ms":
                            d = ""
                            for ii in range(len(i[5])-2):
                                d = d + i[5][ii]
                            d = float(d)
                            f.write(i[0]+","+i[1]+","+i[2]+","+i[3]+","+i[4]+","+str(d/1000)+"s"+"\n")
                        else:
                            f.write(i[0]+","+i[1]+","+i[2]+","+i[3]+","+i[4]+","+i[5]+"\n")
                        
        
        def add_stable(self):
            with open("Gamma_molded.csv", mode = "w") as f:
                for i in self.Gammas:
                    f.write(i[0]+","+i[1]+","+i[2]+","+i[3]+","+i[4]+","+i[5]+"\n")
            
if __name__ == "__main__":
    execute = ToRI()


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/takeh/Research/Xrays_molded.csv'